In [207]:
# Net class: Contains the name of the net and the faults that are associated with it
class Net:
    all = []  # List of all instances
    def __init__(self,name,type = None ,level = None,Gate = None,Gate_inputs = None):
        self.name = int(name)
        self.type = type
        self.faults = ["sa0","sa1"]  # Default faults: Fault Population
        self.level = level
        self.Gate = Gate
        self.Gate_inputs = Gate_inputs
        Net.all.append(self)         # Add instance to list of all instances
        
    def __repr__(self):
        return f"Net_{self.name}" 
    
    def disp_attributes(self):
        # To display all attributes of the instance
        print("-----------------------------------------")
        print(f"Net_{self.name} :")
        print(f"name = {self.name}")
        print(f"type = {self.type}")
        print(f"faults = {self.faults}")
        print(f"level = {self.level}")
        print(f"Gate = {self.Gate}")
        print(f"Gate_inputs = {self.Gate_inputs}")
        print("-----------------------------------------")
    
    @classmethod
    def get_instance(cls,name):
        for instance in cls.all:
            if instance.name == name:
                return instance
        return None
    
    @classmethod
    def check_instance(cls,name):
        for instance in cls.all:
            if instance.name == name:
                return True
        return False



In [208]:
with open("netlist.txt") as f:
    netlist = f.readlines()

# Cleaning the netlist
for i in range(len(netlist)):
    netlist[i] = netlist[i].split()
    netlist[i] = [x for x in netlist[i] if x != '']

print(netlist)

[['AND', '9', '1', '5'], ['FANOUT', '2', '5', '6'], ['AND', '10', '6', '7'], ['FANOUT', '3', '7', '8'], ['INPUT', '1', '2', '3', '4'], ['AND', '11', '4', '8'], ['FANOUT', '10', '12', '13'], ['OR', '14', '9', '12'], ['OR', '15', '13', '11'], ['AND', '16', '14', '15'], ['OUTPUT', '16']]


In [209]:
# Levelise the netlist

# Make list of output nodes , input nodes for given netlist provided 

# find Primary inputs :

output_list = []
input_list = []
primary_inputs = []
primary_output = []



for item in netlist:
    if item[0] == "INPUT":
        for node in item[1:]:
            if Net.check_instance(int(node)):
                n = Net.get_instance(int(node))
                n.type = "PrimaryInput"
                n.level = 0
            else :
                Net(node,"PrimaryInput",0)
            primary_inputs.append(node)
            

    elif item[0] == "OUTPUT":
        if Net.check_instance(int(item[1])):
            n = Net.get_instance(int(item[1]))
            n.type = "PrimaryOutput"
        else:
            Net(item[1],"PrimaryOutput")

        primary_output.append(item[1])
        

    elif item[0] == "FANOUT":
        for node in item[2:]:
            if Net.check_instance(int(node)):
                n = Net.get_instance(int(node))
                n.type = "FanoutWire"
            else:
                Net(node,"FanoutWire")



    elif item[0] in ["AND","OR","NAND","NOR","NOT"]:

        output_list.append(item[1])

        # Handling output node 
        if Net.check_instance(int(item[1])):
            n = Net.get_instance(int(item[1]))
            if n.type == "PrimaryOutput":
                pass
        else:
            Net(item[1],"wire")
        Net.get_instance(int(item[1])).Gate_inputs = item[2:]
        Net.get_instance(int(item[1])).Gate = item[0]

        # Handling input nodes
        for node in item[2:]:
            if Net.check_instance(int(node)):
                n = Net.get_instance(int(node))
                if n.type in ["FanoutWire","PrimaryInput"]:
                    pass
            else:
                Net(node,"wire")

            
            input_list.append(node)
    

input_list = [ node for node in input_list if node not in primary_inputs]

print(output_list)
print(input_list)
print(primary_output)
print(primary_inputs)
print(Net.all)


for obj in Net.all:
    obj.disp_attributes()


['9', '10', '11', '14', '15', '16']
['5', '6', '7', '8', '9', '12', '13', '11', '14', '15']
['16']
['1', '2', '3', '4']
[Net_9, Net_1, Net_5, Net_6, Net_10, Net_7, Net_8, Net_2, Net_3, Net_4, Net_11, Net_12, Net_13, Net_14, Net_15, Net_16]
-----------------------------------------
Net_9 :
name = 9
type = wire
faults = ['sa0', 'sa1']
level = None
Gate = AND
Gate_inputs = ['1', '5']
-----------------------------------------
-----------------------------------------
Net_1 :
name = 1
type = PrimaryInput
faults = ['sa0', 'sa1']
level = 0
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_5 :
name = 5
type = FanoutWire
faults = ['sa0', 'sa1']
level = None
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_6 :
name = 6
type = FanoutWire
faults = ['sa0', 'sa1']
level = None
Gate = None
Gate_inputs = None
-----------------------------------------
--------------

In [210]:
# Levelise the netlist

count = int(max(primary_inputs))+1
curr_level = 0

while count <= int(primary_output[0]):
    if str(count) not in output_list:
        obj = Net.get_instance(count)
        obj.level = curr_level
        count += 1
    else:
        obj = Net.get_instance(count)
        inp = obj.Gate_inputs
        inp = [int(x) for x in inp]
        inp_objects = [Net.get_instance(x) for x in inp]
        inp_obj_levels = [x.level for x in inp_objects]
        print(obj)
        print(inp_obj_levels)
        obj.level = max(inp_obj_levels)+1
        curr_level = obj.level
        count += 1



for obj in Net.all:
    obj.disp_attributes()      
    

Net_9
[0, 0]
Net_10
[0, 0]
Net_11
[0, 0]
Net_14
[1, 1]
Net_15
[1, 1]
Net_16
[2, 2]
-----------------------------------------
Net_9 :
name = 9
type = wire
faults = ['sa0', 'sa1']
level = 1
Gate = AND
Gate_inputs = ['1', '5']
-----------------------------------------
-----------------------------------------
Net_1 :
name = 1
type = PrimaryInput
faults = ['sa0', 'sa1']
level = 0
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_5 :
name = 5
type = FanoutWire
faults = ['sa0', 'sa1']
level = 0
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_6 :
name = 6
type = FanoutWire
faults = ['sa0', 'sa1']
level = 0
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_10 :
name = 10
type = wire
faults = ['sa0', 'sa1']
level = 1
Gate = AND
Gate_inputs = ['6', '7']
-------------------------------------

In [211]:
print(Net.all)

x = Net.get_instance(1)

print(Net.check_instance(1))
print(Net.check_instance(5))

[Net_9, Net_1, Net_5, Net_6, Net_10, Net_7, Net_8, Net_2, Net_3, Net_4, Net_11, Net_12, Net_13, Net_14, Net_15, Net_16]
True
True


In [212]:
import random

# Dominance Implementation

def DominanceCollapsing(obj):

    if obj.Gate == "AND":

        if "sa0" in obj.faults:
            obj.faults.remove("sa0")
        if "sa1" in obj.faults:
            obj.faults.remove("sa1")

        inp_obj = []    
        inps = obj.Gate_inputs

        for inp in inps:
            inp_obj.append(Net.get_instance(int(inp)))

            # Getting wire that is attached to a gate : Retain faults only for this gate 
        retain_fault_obj = [obj for obj in inp_obj if obj.type == "wire"]


        
        # if more gate are inputs: retain first one and remove faults from others
        

        if len(retain_fault_obj) != 0:
            retain_fault_obj = retain_fault_obj[0]
            inp_obj.remove(retain_fault_obj)
        else:
            inp_obj.remove(inp_obj[0])

        # Now collapse the faults of the remaining objects
        for obj in inp_obj:
            if "sa0" in obj.faults:
                obj.faults.remove("sa0")

    
    if obj.Gate == "OR":
        if "sa0" in obj.faults:
            obj.faults.remove("sa0")
        if "sa1" in obj.faults:
            obj.faults.remove("sa1")

        inp_obj = []    
        inps = obj.Gate_inputs

        for inp in inps:
            inp_obj.append(Net.get_instance(int(inp)))

            # Getting wire that is attached to a gate : Retain faults only for this gate 
        retain_fault_obj = [obj for obj in inp_obj if obj.type == "wire"]


        
        # if more gate are inputs: retain first one and remove faults from others
        


        if len(retain_fault_obj) != 0:
            retain_fault_obj = retain_fault_obj[0]
            inp_obj.remove(retain_fault_obj)
        else:
            inp_obj.remove(inp_obj[0])

        # Now collapse the faults of the remaining objects
        for obj in inp_obj:
            if "sa1" in obj.faults:
                obj.faults.remove("sa1")

    
    if obj.Gate == "NAND":
        
        if "sa0" in obj.faults:
            obj.faults.remove("sa0")
        if "sa1" in obj.faults:
            obj.faults.remove("sa1")

        inp_obj = []    
        inps = obj.Gate_inputs

        for inp in inps:
            inp_obj.append(Net.get_instance(int(inp)))

            # Getting wire that is attached to a gate : Retain faults only for this gate 
        retain_fault_obj = [obj for obj in inp_obj if obj.type == "wire"]


        
        # if more gate are inputs: retain first one and remove faults from others
        


        if len(retain_fault_obj) != 0:
            retain_fault_obj = retain_fault_obj[0]
            inp_obj.remove(retain_fault_obj)
        else:
            inp_obj.remove(inp_obj[0])

        # Now collapse the faults of the remaining objects
        for obj in inp_obj:
            if "sa0" in obj.faults:
                obj.faults.remove("sa0")



    if obj.Gate == "NOR":
        
        if "sa" in obj.faults:
            obj.faults.remove("sa0")
        if "sa1" in obj.faults:
            obj.faults.remove("sa1")

        inp_obj = []    
        inps = obj.Gate_inputs

        for inp in inps:
            inp_obj.append(Net.get_instance(int(inp)))

            # Getting wire that is attached to a gate : Retain faults only for this gate 
        retain_fault_obj = [obj for obj in inp_obj if obj.type == "wire"]


        
        # if more gate are inputs: retain first one and remove faults from others
        


        if len(retain_fault_obj) != 0:
            retain_fault_obj = retain_fault_obj[0]
            inp_obj.remove(retain_fault_obj)
        else:
            inp_obj.remove(inp_obj[0])

        # Now collapse the faults of the remaining objects
        for obj in inp_obj:
            if "sa1" in obj.faults:
                obj.faults.remove("sa1")


    if obj.Gate == "NOT":

        if "sa" in obj.faults:
            obj.faults.remove("sa0")
        if "sa1" in obj.faults:
            obj.faults.remove("sa1")

        # No input collapsing for NOT gate
    

        







In [213]:
levelList = sorted(Net.all, key=lambda obj: obj.level,reverse=True)
print(levelList)


[Net_16, Net_14, Net_15, Net_9, Net_10, Net_11, Net_12, Net_13, Net_1, Net_5, Net_6, Net_7, Net_8, Net_2, Net_3, Net_4]


In [214]:

print("Populated Fault list for each node :")

for net in levelList:
    print(f"Net : {net.name}, Faults: {net.faults}")

TotalFaultBeforeCollapsing =  len(Net.all)*2


print(f"\nTotal Faults before Collapsing :{TotalFaultBeforeCollapsing}")

print("------------------------------------")



# Fault collapsing
for obj in levelList:
    DominanceCollapsing(obj)


print("Fault list after Dominance Fault Collapsing :\n")

NumFaultsAfterCollapsing = 0

for net in Net.all:

    print(f"Net :{net.name},  Faults: {net.faults}")
    NumFaultsAfterCollapsing += len(net.faults)

print("------------------------------------")   

print(f"Collapse Ratio :{NumFaultsAfterCollapsing}/{TotalFaultBeforeCollapsing}")

Populated Fault list for each node :
Net : 16, Faults: ['sa0', 'sa1']
Net : 14, Faults: ['sa0', 'sa1']
Net : 15, Faults: ['sa0', 'sa1']
Net : 9, Faults: ['sa0', 'sa1']
Net : 10, Faults: ['sa0', 'sa1']
Net : 11, Faults: ['sa0', 'sa1']
Net : 12, Faults: ['sa0', 'sa1']
Net : 13, Faults: ['sa0', 'sa1']
Net : 1, Faults: ['sa0', 'sa1']
Net : 5, Faults: ['sa0', 'sa1']
Net : 6, Faults: ['sa0', 'sa1']
Net : 7, Faults: ['sa0', 'sa1']
Net : 8, Faults: ['sa0', 'sa1']
Net : 2, Faults: ['sa0', 'sa1']
Net : 3, Faults: ['sa0', 'sa1']
Net : 4, Faults: ['sa0', 'sa1']

Total Faults before Collapsing :32
------------------------------------
Fault list after Equivalence Fault Collapsing :

Net :9,  Faults: []
Net :1,  Faults: ['sa0', 'sa1']
Net :5,  Faults: ['sa1']
Net :6,  Faults: ['sa0', 'sa1']
Net :10,  Faults: []
Net :7,  Faults: ['sa1']
Net :8,  Faults: ['sa1']
Net :2,  Faults: ['sa0', 'sa1']
Net :3,  Faults: ['sa0', 'sa1']
Net :4,  Faults: ['sa0', 'sa1']
Net :11,  Faults: []
Net :12,  Faults: ['sa0']